# Leads and lags in Drought Dynamics
**Meterological, Agricultural, and Hydrological Droughts**

![Laura Crocetti et al., 2020[^1]](https://www.researchgate.net/publication/344947662/figure/fig1/AS:951872072450048@1603955395551/Types-of-drought-meteorological-agricultural-hydrological-and-socio-economic-drought.png)

## Overview

In this notebook we will examine the propagation of precipitation deficits, to soil water deficits, and finally there lagged effect on plants. This cascading effect of drought development is important to study in areas where rain-fed agricultural practices are of importance, such as Mozambique.

- Metoereological drought is described as the lack of precipitation and/or increased atmospheric demand (evapotranspiration)
- Agricultural drought is also a physical phenomenon where soils become replete of moisture
- Hydrological drought follows when ongoing drought can then cause deficits in ground and surface reservoirs of 
- Socioeconomic drought impacts society through crop loss, food insecurity, shortages of fresh water resources

The first three phenomena often follow in a cascasiding sequence, whereas all of them can have a direct impact on society.

[^1]: L. Crocetti et al., Earth Observation for agricultural drought monitoring in the Pannonian Basin (southeastern Europe): current state and future directions, Reg Environ Change, vol. 20, no. 4, p. 123, Dec. 2020, doi: [10.1007/s10113-020-01710-w](http://link.springer.com/10.1007/s10113-020-01710-w).


In [ ]:
%matplotlib inline

import hvplot.pandas  # noqa
import matplotlib
import pandas as pd
import statsmodels.formula.api as smf

matplotlib.use("TkAgg")
%run ./src/corr_plots.py

## Motiroing Drought Dynamics

The following dataset combines the HSAF ASCAT 6.25 surface soil moisture with data for precipitation, CHIRPS, and data for vegetation, NDVI (see table).



| Feature           | NDVI (Normalized Difference Vegetation Index)                          | CHIRPS (Climate Hazards Group InfraRed Precipitation with Station data) |
|-------------------|-----------------------------------------------------------------------|-------------------------------------------------------------------------|
| **Description**   | A widely used index to measure the health and vigor of vegetation.    | A high-resolution precipitation dataset that combines satellite and station data. |
| **Data Source**   | Remote sensing, typically from satellite imagery.                     | Satellite infrared data combined with rain gauge station data.            |
| **Calculation**   | (NIR - RED) / (NIR + RED), where NIR is Near-Infrared and RED is Red light reflectance. | Combines data from multiple sources to provide daily precipitation estimates. |
| **Applications**  | - Agriculture: Monitoring crop health and growth.<br>- Ecology: Assessing vegetation cover and changes.<br>- Climate: Studying vegetation response to climate change. | - Drought monitoring and early warning.<br>- Hydrological modeling.<br>- Agricultural planning and risk management. |
| **Advantages**     | - Simple and quick to calculate.<br>- Provides a standardized measure of vegetation health.<br>- Useful for temporal and spatial comparisons. | - High spatial and temporal resolution.<br>- Integrates multiple data sources for better accuracy.<br>- Useful for regions with sparse rain gauge networks. |
| **Limitations**   | - Sensitive to atmospheric conditions and soil background.<br>- May require calibration for different vegetation types.<br>- Does not distinguish between different types of vegetation. | - Dependent on the quality and availability of input data.<br>- May have errors in areas with complex terrain or dense vegetation.<br>- Requires validation with ground truth data. |
| **Use Cases**     | - Detecting areas of stressed or unhealthy vegetation.<br>- Monitoring deforestation and land use changes.<br>- Assessing the impact of droughts and floods on vegetation. | - Providing real-time precipitation data for disaster response.<br>- Supporting water resource management and irrigation planning.<br>- Enhancing climate models and weather forecasting. |

In short NDVI indicates plant health with assumption that plants become less green when they wilt when deprived from water, with the caviat that the signal depends higly on vegetation types, soil background,, and atmospheric conditions. CHIRPS measures precipitatiomn but can be affected

In [ ]:
%run ./src/download_path.py

url = make_url("drought_indices-6_25_dekadal.csv")  # noqa
df = (
    pd.read_csv(
        url,
        index_col="time",
        parse_dates=True,
    )
    .dropna()
    .sort_index()
)

df

In [ ]:
df.hvplot.line(
    x="time",
    y=["precip", "surface_soil_moisture", "ndvi"],
    subcoordinate_y=True,
    groupby=["name"],
    frame_width=800,
)

In [ ]:
df_buzi = df[df.name == "Buzi"]
df_buzi

In [ ]:
mod_precip = smf.ols(formula="ndvi ~ precip", data=df_buzi)
res_precip = mod_precip.fit()

print("Parameters: \n", res_precip.params, sep="")
print("R2: ", res_precip.rsquared)

In [ ]:
df_buzi = df[df.name == "Buzi"]
mod_ssm = smf.ols(formula="ndvi ~ surface_soil_moisture", data=df_buzi)
res_ssm = mod_ssm.fit()

print("Parameters: \n", res_ssm.params, sep="")
print("R2: ", res_ssm.rsquared)

In [ ]:
results = {"precip": res_precip, "surface_soil_moisture": res_ssm}
plot_predicted_values(  # noqa
    df_buzi,
    results,
    figsize=(10, 5),
)

In [ ]:
plot_step_corr(df_buzi, "ndvi", length=72)  # noqa

In [ ]:
plot_step_corr(df_buzi, "ndvi", "precip", length=72)  # noqa

In [ ]:
plot_step_corr(df_buzi, "ndvi", "surface_soil_moisture", length=72)  # noqa

Auto


Cross

- NDVI lags precipiations with 6 dekads (or ~ 2 months)
- NDVI lags ssm with 3 dekads (or ~ 1 month)



In [ ]:
df_buzi = df[df.name == "Buzi"]
df_buzi_shift = df_buzi.assign(
    surface_soil_moisture=df_buzi.loc[:, "surface_soil_moisture"].shift(3),
    precip=df_buzi.loc[:, "precip"].shift(6),
).dropna()

df_buzi_shift

In [ ]:
mod_precip_shift = smf.ols(formula="ndvi ~ precip", data=df_buzi_shift)
res_precip_shift = mod_precip_shift.fit()

print("Parameters: \n", res_precip_shift.params, sep="")
print("R2: ", res_precip_shift.rsquared)

In [ ]:
mod_ssm_shift = smf.ols(formula="ndvi ~ surface_soil_moisture", data=df_buzi_shift)
res_ssm_shift = mod_ssm_shift.fit()

print("Parameters: \n", res_ssm_shift.params, sep="")
print("R2: ", res_ssm_shift.rsquared)

In [ ]:
results_shift = {"precip": res_precip_shift, "surface_soil_moisture": res_ssm_shift}
plot_predicted_values(  # noqa
    df_buzi_shift,
    results_shift,
    figsize=(10, 5),
    suffix=["(shifted 2 months)", "(shifted 1 month)"],
)